In [129]:
import scanpy as sc
import decoupler as dc

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler

sc.settings.verbosity = 3
sc.set_figure_params(figsize=(4, 4))
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

# Read data and restore `.raw`

In [130]:
# adata = sc.datasets.pbmc3k_processed()
adata = sc.read_h5ad("data/pbmc3k.h5ad")
adata

AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [131]:
adata = adata.raw.to_adata()
adata.layers["log_counts"] = adata.X.copy()
adata

AnnData object with n_obs × n_vars = 2638 × 13714
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    layers: 'log_counts'
    obsp: 'distances', 'connectivities'

In [132]:
model = dc.get_progeny(organism='human', top=100)
model

,source,target,weight,p_value
0,Androgen,TMPRSS2,11.490631,0.000000e+00
1,Androgen,NKX3-1,10.622551,2.242078e-44
2,Androgen,MBOAT2,10.472733,4.624285e-44
3,Androgen,KLK2,10.176186,1.944414e-40
4,Androgen,SARG,11.386852,2.790209e-40
...,...,...,...,...
1395,p53,CCDC150,-3.174527,7.396252e-13
1396,p53,LCE1A,6.154823,8.475458e-13
1397,p53,TREM2,4.101937,9.739648e-13
1398,p53,GDF9,3.355741,1.087433e-12


# Run decoupler with log_counts

use `dc.run_mlm`

In [133]:
adata.X = adata.layers["log_counts"].copy()
dc.run_mlm(mat=adata, net=model, source='source', target='target', weight='weight', use_raw=False, verbose=True)

1 features of mat are empty, they will be removed.
Running mlm on mat with 2638 samples and 13713 targets for 14 sources.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.52it/s]


In [134]:
adata.obsm["log_counts_mlm_estimate"] = adata.obsm["mlm_estimate"]
adata.obsm["log_counts_mlm_pvals"] = adata.obsm["mlm_pvals"]

In [135]:
adata.obsm["log_counts_mlm_estimate"].iloc[:5, :5]

,Androgen,EGFR,Estrogen,Hypoxia,JAK-STAT
AAACATACAACCAC-1,-0.250004,1.082446,-0.283717,0.018847,-1.101526
AAACATTGAGCTAC-1,0.055699,1.808353,-0.639859,0.547163,0.053690
AAACATTGATCAGC-1,-1.300669,1.093902,-1.423962,0.862988,0.770645
AAACCGTGCTTCCG-1,-1.108973,0.623916,-0.533218,0.456434,5.369927
AAACCGTGTATGCG-1,-0.020885,-0.934698,0.113905,0.480329,2.008395


# Run decoupler with zscaled_counts

use `dc.run_mlm`, too

In [136]:
adata.X = StandardScaler().fit_transform(np.expm1(adata.layers["log_counts"].toarray()))
dc.run_mlm(mat=adata, net=model, source='source', target='target', weight='weight', use_raw=False, verbose=True)

1 features of mat are empty, they will be removed.
Running mlm on mat with 2638 samples and 13713 targets for 14 sources.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.33it/s]


In [137]:
adata.obsm["zscaled_mlm_estimate"] = adata.obsm["mlm_estimate"]
adata.obsm["zscaled_counts_mlm_pvals"] = adata.obsm["mlm_pvals"]

In [138]:
adata.obsm["zscaled_mlm_estimate"].iloc[:5, :5]

,Androgen,EGFR,Estrogen,Hypoxia,JAK-STAT
AAACATACAACCAC-1,1.028017,0.372965,-0.088143,0.323772,-1.788776
AAACATTGAGCTAC-1,0.399224,1.233052,-0.272808,-0.306251,-1.185899
AAACATTGATCAGC-1,-0.792074,1.111080,-1.434491,0.127331,-0.441286
AAACCGTGCTTCCG-1,-0.761711,-0.623148,-0.047732,0.697356,2.724582
AAACCGTGTATGCG-1,1.166387,-2.516185,0.721634,0.506971,-0.380416


# Run decoupler with zscaled_log_counts

use `dc.run_mlm`, too

In [139]:
adata.X = StandardScaler().fit_transform(adata.layers["log_counts"].toarray())
dc.run_mlm(mat=adata, net=model, source='source', target='target', weight='weight', use_raw=False, verbose=True)

1 features of mat are empty, they will be removed.
Running mlm on mat with 2638 samples and 13713 targets for 14 sources.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.39it/s]


In [140]:
adata.obsm["zscaled_log_counts_mlm_estimate"] = adata.obsm["mlm_estimate"]
adata.obsm["zscaled_log_counts_mlm_pvals"] = adata.obsm["mlm_pvals"]

In [141]:
adata.obsm["zscaled_log_counts_mlm_estimate"].iloc[:5, :5]

,Androgen,EGFR,Estrogen,Hypoxia,JAK-STAT
AAACATACAACCAC-1,1.273499,0.539059,0.150309,0.313245,-2.431031
AAACATTGAGCTAC-1,0.602056,1.708841,-0.327246,-0.015498,-1.374125
AAACATTGATCAGC-1,-0.943165,1.072075,-1.580218,0.084604,-0.379241
AAACCGTGCTTCCG-1,-0.881047,-0.578264,-0.103478,0.852448,3.470078
AAACCGTGTATGCG-1,1.282600,-2.897524,0.950830,0.660910,-0.178160


# The activity correlation computed by log_counts and zcaled_counts

In [142]:
adata.obsm["zscaled_mlm_estimate"].corrwith(adata.obsm["log_counts_mlm_estimate"], axis=0, method="pearson")

Androgen    0.799440
EGFR        0.766055
Estrogen    0.722386
Hypoxia     0.718415
JAK-STAT    0.908440
MAPK        0.716142
NFkB        0.649712
PI3K        0.771973
TGFb        0.818348
TNFa        0.642214
Trail       0.687261
VEGF        0.615875
WNT         0.653141
p53         0.813411
dtype: float64

# The activity correlation computed by log_counts and zcaled_log_counts

In [143]:
adata.obsm["zscaled_log_counts_mlm_estimate"].corrwith(adata.obsm["log_counts_mlm_estimate"], axis=0, method="pearson")

Androgen    0.864412
EGFR        0.830156
Estrogen    0.783813
Hypoxia     0.768761
JAK-STAT    0.965310
MAPK        0.786431
NFkB        0.727709
PI3K        0.835296
TGFb        0.843625
TNFa        0.727131
Trail       0.774298
VEGF        0.668668
WNT         0.718179
p53         0.862346
dtype: float64

# The activity correlation computed by zscaled_counts and zcaled_log_counts

In [144]:
adata.obsm["zscaled_log_counts_mlm_estimate"].corrwith(adata.obsm["zscaled_mlm_estimate"], axis=0, method="pearson")

Androgen    0.958007
EGFR        0.963163
Estrogen    0.963415
Hypoxia     0.974902
JAK-STAT    0.955867
MAPK        0.966529
NFkB        0.930859
PI3K        0.953268
TGFb        0.985438
TNFa        0.927248
Trail       0.953857
VEGF        0.981271
WNT         0.955993
p53         0.965798
dtype: float64